<a href="https://colab.research.google.com/github/aditisingh2912/Data-Mining-/blob/main/FP_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import defaultdict
transactions = [
    {'E', 'K', 'M', 'N', 'O', 'Y'},
    {'D', 'E', 'K', 'N', 'O', 'Y'},
    {'A', 'E', 'K', 'M'},
    {'C', 'K', 'M', 'U', 'Y'},
    {'C', 'E', 'I', 'K', 'O'}
]
MIN_SUPPORT = 3
MIN_CONFIDENCE = 0.7

item_counts = defaultdict(int)
for transaction in transactions:
    for item in transaction:
        item_counts[item] += 1

frequent_items = {item: count for item, count in item_counts.items() if count >= MIN_SUPPORT}
sorted_items = sorted(frequent_items.items(), key=lambda x: (-x[1], x[0]))
item_order = [item for item, _ in sorted_items]

class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.node_link = None


def build_fp_tree(transactions, frequent_items, item_order):
    root = FPNode(None, 0, None)
    last_node = {}

    for transaction in transactions:
        filtered = [item for item in transaction if item in frequent_items]
        sorted_transaction = sorted(filtered, key=lambda x: item_order.index(x))

        current_node = root
        for item in sorted_transaction:
            if item in current_node.children:
                current_node.children[item].count += 1
            else:
                new_node = FPNode(item, 1, current_node)
                current_node.children[item] = new_node

                if item in last_node:
                    last_node[item].node_link = new_node
                last_node[item] = new_node

            current_node = current_node.children[item]

    return root

def find_frequent_itemsets(root, min_support, prefix, frequent_itemsets):
    item_nodes = {}

    def traverse(node):
        if node.item:
            if node.item not in item_nodes:
                item_nodes[node.item] = []
            item_nodes[node.item].append(node)
        for child in node.children.values():
            traverse(child)

    traverse(root)

    for item in reversed(item_order):
        if item not in item_nodes:
            continue

        new_prefix = prefix | {item}
        support = sum(node.count for node in item_nodes[item])

        if support >= min_support:
            frequent_itemsets[frozenset(new_prefix)] = support

        conditional_base = []
        for node in item_nodes[item]:
            path = []
            current = node.parent
            while current and current.item is not None:
                path.append(current.item)
                current = current.parent
            if path:
                conditional_base.append((path, node.count))

        # Build conditional FP-tree
        cond_counts = defaultdict(int)
        for path, count in conditional_base:
            for item in path:
                cond_counts[item] += count

        cond_frequent = {k: v for k, v in cond_counts.items() if v >= min_support}
        if not cond_frequent:
            continue

        cond_item_order = sorted(cond_frequent.keys(), key=lambda x: item_order.index(x))
        cond_root = build_fp_tree([set(path) for path, _ in conditional_base], cond_frequent, cond_item_order)

        # Recursively mine conditional tree
        find_frequent_itemsets(cond_root, min_support, new_prefix, frequent_itemsets)

def generate_confidence(frequent_itemsets, min_confidence):
    confidence_rules = {}

    for itemset, support in frequent_itemsets.items():
        if len(itemset) < 2:
            continue

        for item in itemset:
            antecedent = frozenset([item])
            consequent = itemset - antecedent

            if consequent and antecedent in frequent_itemsets:
                confidence = support / frequent_itemsets[antecedent]
                if confidence >= min_confidence:
                    confidence_rules[(antecedent, consequent)] = confidence

    return confidence_rules

def main():
    print("Frequent 1-Itemsets:", set(frequent_items.keys()))
    root = build_fp_tree(transactions, frequent_items, item_order)
    frequent_itemsets = {}
    find_frequent_itemsets(root, MIN_SUPPORT, frozenset(), frequent_itemsets)
    confidence_rules = generate_confidence(frequent_itemsets, MIN_CONFIDENCE)

    frequent_2_itemsets = {k: v for k, v in frequent_itemsets.items() if len(k) == 2}
    frequent_3_itemsets = {k: v for k, v in frequent_itemsets.items() if len(k) == 3}
    print("Frequent 2-Itemsets:", set(frequent_2_itemsets.keys()))
    print("Frequent 3-Itemsets:", set(frequent_3_itemsets.keys()))

    print("\nAssociation Rules (Confidence ≥ 0.7):")
    for (antecedent, consequent), conf in confidence_rules.items():
        print(f"{set(antecedent)} → {set(consequent)}: {conf:.2f}")

if __name__ == "__main__":
    main()

Frequent 1-Itemsets: {'M', 'Y', 'O', 'E', 'K'}
Frequent 2-Itemsets: {frozenset({'K', 'Y'})}
Frequent 3-Itemsets: set()

Association Rules (Confidence ≥ 0.7):
{'Y'} → {'K'}: 1.00



2. Compute the frequent itemsets and the strong association rules for the below dataset by
assuming the support threshold of 4 and confidence of 60%.


In [ ]:
from collections import defaultdict, namedtuple
from itertools import combinations

class FPTreeNode:
    def __init__(self, item, count=1, parent=None):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

    def increase_count(self, count=1):
        self.count += count

    def display(self, indent=1):
        print(' ' * indent, self.item, ' ', self.count)
        for child in self.children.values():
            child.display(indent + 2)

class FPTree:
    def __init__(self, transactions, min_support):
        self.min_support = min_support
        self.header_table = defaultdict(list)
        self.root = FPTreeNode(None)
        self.item_support = defaultdict(int)

        for transaction in transactions:
            for item in transaction:
                self.item_support[item] += 1

        self.item_support = {k: v for k, v in self.item_support.items() if v >= min_support}

        for transaction in transactions:
            sorted_items = [item for item in sorted(transaction, key=lambda x: self.item_support.get(x, 0), reverse=True) if item in self.item_support]
            self.insert_tree(sorted_items, self.root)

    def insert_tree(self, items, node):
        if not items:
            return
        first, *rest = items
        if first in node.children:
            node.children[first].increase_count()
        else:
            new_node = FPTreeNode(first, 1, node)
            node.children[first] = new_node
            if first in self.header_table:
                last_node = self.header_table[first][-1]
                last_node.link = new_node
            self.header_table[first].append(new_node)

        self.insert_tree(rest, node.children[first])

    def mine_patterns(self, prefix=set()):
        patterns = {}
        for item in sorted(self.header_table, key=lambda x: self.item_support[x]):
            new_prefix = prefix | {item}
            patterns[frozenset(new_prefix)] = self.item_support[item]
            conditional_tree = self.build_conditional_tree(item)
            if conditional_tree:
                patterns.update(conditional_tree.mine_patterns(new_prefix))
        return patterns

    def build_conditional_tree(self, item):
        paths = []
        node = self.header_table[item][0] if self.header_table[item] else None
        while node:
            path, current = [], node.parent
            while current and current.item is not None:
                path.append(current.item)
                current = current.parent
            path.reverse()
            if path:
                paths.append((path, node.count))
            node = node.link

        conditional_transactions = []
        for path, count in paths:
            conditional_transactions.extend([path] * count)

        return FPTree(conditional_transactions, self.min_support) if conditional_transactions else None

def generate_association_rules(patterns, transactions, min_confidence):
    def compute_confidence(A, B):
        count_A = sum(1 for t in transactions if A.issubset(t))
        count_AB = sum(1 for t in transactions if A.union(B).issubset(t))
        return (count_AB / count_A) * 100 if count_A else 0

    strong_rules = []
    for itemset in patterns:
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = set(antecedent)
                consequent = itemset - antecedent
                confidence = compute_confidence(antecedent, consequent)
                if confidence >= min_confidence:
                    strong_rules.append((antecedent, consequent, confidence))
    return strong_rules

transactions = [
    {'Apple', 'Banana'},
    {'Cherry', 'Apple'},
    {'Banana', 'Cherry', 'Date'},
    {'Apple', 'Date', 'Fig'},
    {'Banana', 'Cherry'},
    {'Apple', 'Banana', 'Date'},
    {'Cherry', 'Fig'},
    {'Banana', 'Apple'},
    {'Cherry', 'Date'},
    {'Apple', 'Banana'}
]

min_support = 4
min_confidence = 60
fp_tree = FPTree(transactions, min_support)
fp_patterns = fp_tree.mine_patterns()
print("Frequent Patterns:", fp_patterns)
strong_rules = generate_association_rules(fp_patterns, transactions, min_confidence)
print("Strong Association Rules:", strong_rules)


Frequent Patterns: {frozenset({'Date'}): 4, frozenset({'Cherry'}): 5, frozenset({'Apple'}): 6, frozenset({'Banana'}): 6, frozenset({'Apple', 'Banana'}): 4}
Strong Association Rules: [({'Apple'}, frozenset({'Banana'}), 66.66666666666666), ({'Banana'}, frozenset({'Apple'}), 66.66666666666666)]
